# [Author : TsukiSama9292 - Github Link](https://github.com/tsukisama9292)
## [References - Link](https://ianwu.tw/press/topic/command_line_program/yt-dlp.html#%E4%BB%8B%E7%B4%B9)

# Dependency Installation

In [ ]:
### Install ffmpeg on your own computer; Colab already has ffmpeg installed

# Windows : https://vocus.cc/article/64701a2cfd897800014daed0

# macOS : https://blog.csdn.net/csdn_yudong/article/details/129753111

# Linux : directly run the following sudo commands:
# !sudo apt update
# !sudo apt install ffmpeg

In [ ]:
%pip install -q yt-dlp
%pip install -q -U gradio

# Using GUI as the download tool. The default is the highest video and audio quality.  

In [ ]:
import yt_dlp
import threading
import queue
import os
import zipfile
import shutil
from datetime import datetime
import gradio as gr
# Global variables or global state used to store playlist information
playlist_info = {
    'total_videos': 0,
    'downloaded_videos': 0,
}
count = True
ydl_opts = None  # Initialize ydl_opts
### 616 is the 1080p-Premium channel
resolution_option = {
    'Auto (Highest Resolution)': ['bestvideo'],
    '8K(4320)': ['571'],
    '4K(2160)': ['628', '315', '625', '313', '401'],
    '2K(1440)': ['623', '308', '620', '271', '400'],
    'FHD(1080)': ['616', '312', '617', '299', '303', '270', '614', '137', '399', '248'],
    'HD(720)': ['298', '612', '302', '136', '609', '398', '247', '311', '232'],
    '480': ['231', '606', '135', '244', '397'],
    '360': ['230', '605', '18', '134', '243', '396'],
    '240': ['604', '229', '133', '242', '395'],
    '144': ['603', '269', '278', '160', '394'],
}

resolution_high = {
    'Auto (Highest Resolution)': ['bestvideo'],
    '8K(4320)': ['571'],
    '4K(2160)': ['628', '315', '625', '313', '401'],
    '2K(1440)': ['623', '308', '620', '271', '400'],
    'FHD(1080)': ['616', '312', '617', '299', '303', '270', '614', '137', '399', '248'],
    'HD(720)': ['298', '612', '302', '136', '609', '398', '247', '311', '232'],
    '480': ['231', '606', '135', '244', '397'],
    '360': ['230', '605', '18', '134', '243', '396'],
    '240': ['604', '229', '133', '242', '395'],
    '144': ['603', '269', '278', '160', '394'],
}


resolution_avg={
    'Auto (Highest Resolution)': ['571', '315', '313', '401', '628', '625', '308', '271', '400', '623', '620', '299', '617', '303', '137', '614', '399', '248', '312', '270', '616', '311', '612', '298', '302', '232', '609', '136', '398', '247', '135', '606', '244', '397', '231', '18', '134', '605', '243', '396', '230', '133', '299', '242', '395', '604', '278', '269', '160', '394', '603'],
    '8K(4320)':['571'],
    '4K(2160)':['315','313','401','628','625'],
    '2K(1440)':['308','271','400','623','620'],
    'FHD(1080)':['299','617','303','137','614','399','248','312','270','616'],
    'HD(720)':['311','612','298','302','232','609','136','398','247'],
    '480':['135','606','244','397','231'],
    '360':['18','134','605','243','396','230'],
    '240':['133','299','242','395','604'],
    '144':['278','269','160','394','603'],
}

resolution_low={
    'Auto (Highest Resolution)': ['571', '401', '313', '625', '315', '628', '400', '271', '620', '308', '623', '248', '399', '137', '614', '270', '303', '299', '617', '312', '616', '247', '398', '136', '609', '232', '302', '298', '612', '311', '397', '244', '135', '606', '231', '396', '243', '134', '18', '605', '230', '395', '242', '133', '229', '604', '394', '160', '278', '269', '603'],
    '8K(4320)': ['571'],
    '4K(2160)': ['401', '313', '625', '315', '628'],
    '2K(1440)': ['400', '271', '620', '308', '623'],
    'FHD(1080)': ['248', '399', '137', '614', '270', '303', '299', '617', '312','616'],
    'HD(720)': ['247', '398', '136', '609', '232', '302', '298', '612', '311'],
    '480': ['397', '244', '135', '606', '231'],
    '360': ['396', '243', '134', '18', '605', '230'],
    '240': ['395', '242', '133', '229', '604'],
    '144': ['394', '160', '278', '269', '603'],
}

def get_playlist_video_urls(YOUTUBE_URL):
    # Initialize yt-dlp
    ydl = yt_dlp.YoutubeDL({'format': 'best'})
    
    # Get playlist information
    info = ydl.extract_info(YOUTUBE_URL, download=False)
    
    # Check if it's a playlist
    if 'entries' in info:
        # Extract YouTube links of all videos
        video_links = [entry['webpage_url'] for entry in info['entries'] if 'webpage_url' in entry]
        return video_links
    else: # If there's only one video, return directly
        return [YOUTUBE_URL]

def get_video_quantity(YOUTUBE_URL):
    # Initialize yt-dlp
    ydl = yt_dlp.YoutubeDL({'format': 'best'})
    # Get playlist information
    info = ydl.extract_info(YOUTUBE_URL, download=False)
    if 'entries' in info:
        playlist_info['total_videos'] = len(info['entries'])
    else:
        playlist_info['total_videos'] = 1

# Define progress callback function
def progress_hook(q):
    def hook(d):
        # Update the download progress of the playlist
        global playlist_info
        global count
        if d['status'] == 'downloading':
            speed = d.get('speed', 0)
            speed_mb = speed / 1024 / 1024 if speed is not None else 0
            q.put({
                'Status': 'downloading',
                'Downloaded Size MB': d['downloaded_bytes'] / 1024 / 1024,
                'Current Video Total Size MB': d.get('total_bytes', 0) / 1024 / 1024,
                'Network Speed MB/sec': speed_mb,
                'Estimated Time Remaining sec': d.get('eta', 0),
                'Completion Progress': f"{playlist_info['downloaded_videos']} / {playlist_info['total_videos']}"
            })
        elif d['status'] == 'finished':
            if(count):
                count = False
            else:
                count = True
                playlist_info['downloaded_videos'] += 1
            q.put({
                'Status': 'single_download_complete',
                'File Path': d['filename'],
                'Completion Progress': f"{playlist_info['downloaded_videos']} / {playlist_info['total_videos']}"
            })
    return hook


# Define the download function
def download_Run(ydl_opts, url, q):
    ydl_opts['progress_hooks'] = [progress_hook(q)]
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def download_Thread(ydl_opts,url):
    # Create a Queue to pass progress messages
    q = queue.Queue()

    # Execute the download function in a separate thread
    thread = threading.Thread(target=download_Run, args=(ydl_opts, url, q))
    thread.start()

    # Display the download progress in real-time
    while thread.is_alive() or not q.empty():
        try:
            progress = q.get(timeout=1)
            yield progress
        except queue.Empty:
            pass
        
def re_ydl_opts_resolution(YOUTUBE_URL,resolution,out_path):
    bool_list = [False] * len(resolution)
    global ydl_opts
    # Initialize yt-dlp
    ydl = yt_dlp.YoutubeDL({'format': 'best'})
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': out_path,
        'postprocessors': [
            {
                'key': 'FFmpegMetadata'
            }
        ]
    }
    # Check if the quality format is available
    info = ydl.extract_info(YOUTUBE_URL, download=False)
    formats = info.get('formats', [])
    for format in formats:
        for index,resolution_id in enumerate(resolution):  
            if format.get('format_id') == resolution_id:
                bool_list[index]=True
    for index,bool_i in enumerate(bool_list):
        if(bool_i): 
            ydl_opts = {
            'format': f'{resolution[index]}+bestaudio/best',
            'outtmpl': out_path,
            'postprocessors': [
                {
                    'key': 'FFmpegMetadata'
                }
            ]
            }
            break

def download_video(YOUTUBE_URL, do_zip,resolution):
    global ydl_opts
    if(do_zip):
        out_path='Downloads_TMP/%(title)s.%(ext)s'
        links = get_playlist_video_urls(YOUTUBE_URL)
        for link in links:
            re_ydl_opts_resolution(link,resolution,out_path)
            yield from download_Thread(ydl_opts,link)
        if has_files('Downloads_TMP'):
            zip_files('Video')
            yield "Video download complete, please check the path to the Downloads_ZIP/Video folder"
        else:
            yield "Invalid URL input, unable to retrieve video"
    else:
        out_path='Downloads/Video/%(title)s.%(ext)s'
        links = get_playlist_video_urls(YOUTUBE_URL)
        for link in links:
            re_ydl_opts_resolution(link,resolution,out_path)
            yield from download_Thread(ydl_opts,link)
        if has_files('Downloads/Video'):
            yield "Video download completed, please check the path to the Downloads/Video folder"
        else:
            yield "Invalid URL input, unable to retrieve video"

def download_audio(YOUTUBE_URL, do_zip):
    if(do_zip):
        ydl_opts = {
        'format': 'bestaudio/best',  
        'outtmpl': 'Downloads_TMP/%(title)s.%(ext)s', 
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        },{
            'key': 'FFmpegMetadata'
        }],
        }
        yield from download_Thread(ydl_opts,YOUTUBE_URL)
        if has_files('Downloads_TMP'):
            zip_files('Audio')
            yield "Audio download completed, please check the path to the Downloads_ZIP/Audio folder"
        else:
            yield "Invalid URL input, unable to retrieve audio"
    else:
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': 'Downloads/Audio/%(title)s.%(ext)s',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            },{
            'key': 'FFmpegMetadata'
        }],
        }
        yield from download_Thread(ydl_opts,YOUTUBE_URL)
        if has_files('Downloads/Audio'):
            yield "Audio download completed, please check the path to the Downloads/Audio folder"
        else:
            yield "Invalid URL input, unable to retrieve audio"
            
    
def has_files(folder_path):
    for _, _, files in os.walk(folder_path):
        if files:
            return True
    return False

def zip_files(Type):
    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day
    hour = now.hour
    minute = now.minute
    second = now.second
    folder_to_zip = 'Downloads_TMP'
    if has_files(folder_to_zip):
        zip_folder = 'Downloads_ZIP'
        if not os.path.exists(zip_folder):
            os.makedirs(zip_folder)
        
        zip_filename = f'{zip_folder}/{Type}_{year}_{month}_{day}_{hour}_{minute}_{second}.zip'
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for root, dirs, files in os.walk(folder_to_zip):
                for file in files:
                    zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_to_zip))
        folder_to_zip = 'Downloads_TMP'
        if os.path.exists(folder_to_zip):
            shutil.rmtree(folder_to_zip)


def gradio_interface(function_choice, YOUTUBE_URL, do_zip,quality,resolution):
    global resolution_option
    global resolution_high
    global resolution_avg
    global resolution_low
    global playlist_info
    global count
    
    playlist_info = {
        'total_videos': 0,
        'downloaded_videos': 0,
    }
    get_video_quantity(YOUTUBE_URL)
    count = True
    folder_to_zip = 'Downloads_TMP'
    if os.path.exists(folder_to_zip):
        shutil.rmtree(folder_to_zip)
    if quality == "Highest":
        resolution_option = resolution_high
    elif quality == "Average":
        resolution_option = resolution_avg
    elif quality == "Lowest":
        resolution_option = resolution_low
    if function_choice == "Download Video":
        yield from download_video(YOUTUBE_URL, do_zip, resolution_option[resolution])
    elif function_choice == "Download Audio":
        yield from download_audio(YOUTUBE_URL, do_zip)

def update_quality(function_choice):
    if function_choice == "Download Video":
        return gr.update(visible=True), gr.update(visible=True) 
    else:
        return gr.update(visible=False), gr.update(visible=False) 
# Use Gradio Blocks to create the interface
with gr.Blocks() as iface:
    function_choice = gr.Radio(["Download Video", "Download Audio"], label="Select Function")
    YOUTUBE_URL = gr.Textbox(label="Enter YouTube Playlist or Video URL")
    do_zip = gr.Checkbox(label="Compress to ZIP")
    quality = gr.Radio(["Highest", "Average", "Lowest"], label="Select Quality (Higher quality means larger file size)", visible=False)
    resolution = gr.Dropdown(list(resolution_option.keys()), label="Select Resolution", visible=False)
    
    function_choice.change(fn=update_quality, inputs=function_choice, outputs=[quality, resolution])
    
    iface_interface = gr.Interface(
        fn=gradio_interface,
        inputs=[
            function_choice,
            YOUTUBE_URL,
            do_zip,
            quality,
            resolution
        ],
        outputs="text"
    )

# On personal computers, you can set share=False, but on Colab, it must be share=True
iface.queue().launch(share=True)
# 4K 60fps video link (for testing): https://youtu.be/3aRJtJRa434
# Playlists and videos should not use share links (ending in share), use regular URLs like: https://www.youtube.com/playlist?list=PLm2oaJc6zPi1ZJyHaGid4P3sCke4b0qfL